# Evaluación PYTHON + DATA
### Miguel Ángel Fuentes Valenzuela | miguel.fuentes.valenzuela@nter.es

In [1]:
import requests as req
import zipfile as zipf
import pandas as pd
import logging as log

#Librerias extra que también uso:
from unidecode import unidecode
import sys
import os
import io

output_path = "output"
api_url = "https://portal.mineco.gob.es/economiayempresa/EconomiaInformesMacro/Documents/bdsicecsv.zip"

## Primero, configuraremos el módulo Logging que vamos a usar

In [4]:
ev_logger = log.getLogger("ev_logger")

# Configurar el modo del logger a 'DEBUG' y los manejadores de ficheros que almacenarán los logs
ev_logger.setLevel(log.DEBUG)


# Dar formato a los fileHandler's mediante un objeto logging.Formatter y el FileHandler
formateador = log.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fhand_Critical = log.FileHandler("logFile.log")
fhand_Critical.setFormatter(formateador)

# Especificar nivel de los fileHandler's
fhand_Critical.setLevel(log.DEBUG)
# Aplicar el fileHandler a los logger's
ev_logger.addHandler(fhand_Critical)

## A continuación, obtendremos de la API el .zip requerido
### Para ello, usaremos una petición GET (y aprovecho una función que ya tenía guardada del Día 9 (uso de APIs))

In [7]:
def conectarYComprobar(url: str, parm: dict=None, headrs: dict=None):
    '''
    Hace una petición GET a la url indicada; devuelve la respuesta del servidor
    y hace print por pantalla del código HTTP devuelto tras hacer
    la petición.
    '''
    print("Petición enviada! Recbiendo respuesta...")
    if (parm is not None and headrs is not None):
        response = req.get(url, params=parm, headers=headrs, verify=False)
    elif (parm is not None):
        response = req.get(url, parm, verify=False)
    else:
        response = req.get(url, verify=False) # Debemos utilizar verify=False, porque sino nos obligará a verificar el certificado SSL en este caso
    
    print(f"Codigo de respuesta para {url}: {response.status_code}")
    return response


La API es la siguiente:<b> https://portal.mineco.gob.es/economiayempresa/EconomiaInformesMacro/Documents/bdsicecsv.zip </b>

In [10]:
resp = None

try:
    ev_logger.log(log.WARNING, f"Intentando conectar a la API [{api_url}]...")
    resp = conectarYComprobar(api_url)
except:
    ev_logger.log(log.ERROR, f"FALLÓ AL CONECTAR! Intento de conexión a [{api_url}]")
    sys.exit(1)

try:
    ev_logger.log(log.WARNING, f"Datos encontrados! Comenzando extracción del fichero ZIP...")
    #Ahora, usare zipfile para obtener los ficheros CSV de la descarga
    zipDescarga = zipf.ZipFile(io.BytesIO(resp.content))
    #Crear la carpeta si no existe antes
    if ("output" not in os.listdir()):
        os.mkdir("output")
    zipDescarga.extractall("output/") #extraer con ZipFile en la carpeta indicada
    ev_logger.log(log.WARNING, "Hecho! Ficheros extraidos correctamente!")
except:
    ev_logger.log(log.ERROR, f"FALLÓ AL EXTRAER!")
    sys.exit(1)


Petición enviada! Recbiendo respuesta...


C:\Users\miguel.fuentes.vale\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portal.mineco.gob.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\miguel.fuentes.vale\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bdsice.mineco.gob.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Codigo de respuesta para https://portal.mineco.gob.es/economiayempresa/EconomiaInformesMacro/Documents/bdsicecsv.zip: 200


## Ahora, obtendremos los CSV pertenecientes a los códigos que necesitamos

In [12]:
cons_cemento = "236000.CSV"
cons_viviendas = "232020.CSV"
remun_personal = "715100.CSV"
pobl_parada = "170000n.CSV"

ficheros = [cons_cemento, cons_viviendas, remun_personal, pobl_parada]
ficheros_dir = os.listdir("output/")
# comprobar que existen
for fichero in ficheros:
    if fichero not in ficheros_dir:
        print("El fichero necesario " + fichero + " no está en el directorio de salida!")

#Mediante la librería Pandas, los almacenaremos en diferentes dataframes
## De esta manera, será mas facil limpiar datos y sacar info. relevante

In [15]:
df_listado = list()
for fichero in ficheros:
    df = pd.read_csv("output/" + fichero, sep=";", encoding="ISO-8859-1", na_values="null")
    df.name = fichero
    df_listado.append(df)

#comprobar que se han guardado correctamente; Mostrando las 10 primeras filas de cada df
for df in df_listado:
    print(df.tail(10))

      annus   mes               observaciones  código  título  unidad  fuente  \
716  2023.0   9.0                  1138,35300     NaN     NaN     NaN     NaN   
717  2023.0  10.0                  1197,29300     NaN     NaN     NaN     NaN   
718  2023.0  11.0                  1290,45000     NaN     NaN     NaN     NaN   
719  2023.0  12.0                  1059,28600     NaN     NaN     NaN     NaN   
720  2024.0   1.0                  1055,77200     NaN     NaN     NaN     NaN   
721  2024.0   2.0                  1168,89000     NaN     NaN     NaN     NaN   
722  2024.0   3.0                  1100,84200     NaN     NaN     NaN     NaN   
723  2024.0   4.0                  1286,08800     NaN     NaN     NaN     NaN   
724  2024.0   5.0                  1311,67900     NaN     NaN     NaN     NaN   
725     NaN   NaN                         NaN     NaN     NaN     NaN     NaN   

     notas  
716    NaN  
717    NaN  
718    NaN  
719    NaN  
720    NaN  
721    NaN  
722    NaN  
723 

## Limpieza y tratado de datos con Pandas

### Primero las columnas:

In [18]:
#Para hacer limpieza respecto a los nombres de columnas, y estandarizarlos.
## Usaré list comprehension, la librería unidecode y luego replace en pandas, para cambiar cualquier nombre de columnas con tilde en su caracter sin tilde
try:
    for df in df_listado:
        columnas = df.columns.values
        columnas_formateadas = [unidecode(nombre) for nombre in columnas] 
        # comprobando si es ascii un string, se sabe si tiene caractéres extraños (tildes, p.ejemplo) o no
        # Info encontrada en: https://stackoverflow.com/questions/21843971/detecting-accents-in-words-python
        # solución de unidecode = https://www.geeksforgeeks.org/transliterating-non-ascii-characters-with-python/
        df.columns = columnas_formateadas
except:
    ev_logger.log(log.ERROR, "FALLO al intentar renombrar columnas")

#comprobar que se han cambiado sus nombres correctamente
for df in df_listado:
    print(df.head(1))

    annus  mes               observaciones    codigo  \
0  1964.0  1.0                   787,10000  236000.0   

                      titulo              unidad                   fuente  \
0  CEMENTO. CONSUMO APARENTE  MILES DE TONELADAS  MINISTERIO DE INDUSTRIA   

                                               notas  
0  https://industria.gob.es/es-es/estadisticas/pa...  
    annus  mes               observaciones    codigo  \
0  2004.0  1.0                 50657,00000  232020.0   

                                             titulo    unidad  \
0  VIVIENDAS INICIADAS. TOTAL (LIBRES Y PROTEGIDAS)  UNIDADES   

                  fuente                                              notas  
0  MINISTERIO DE FOMENTO  Vivienda libre y protegida  (www.mitma.es/el-m...  
    annus   mes               observaciones    codigo  \
0  1975.0  12.0                          ND  715100.0   

                                              titulo             unidad  \
0  ESTADO. CAJA. GASTOS.  OBLIG 

### Despues, limpieza de datos, y agregación de columnas:

In [21]:
# limpiar datos y dar formato correcto a las columnas
for df in df_listado:
    df["annus"] = df["annus"].fillna(0)
    df["annus"] = df["annus"].astype('i')
    df["mes"] = df["mes"].fillna(0)
    df["mes"] = df["mes"].astype('i')
    df["observaciones"] = df["observaciones"].fillna(0)

    #Agregar las columnas requeridas con los datos ya limpios
    #La fecha sera un string, tal y como se pide (paso las fechas a strings UNICODE)
    df["fecha"] = df["mes"].astype('U') + "/" + df["annus"].astype('U')
    #"valor" sera los valores de 'observaciones' pero con un solo decimal. Los valores que no sean digitos se transformaran a 0.00
    #Convertir todos los 'ND' a valor 0 numérico
    df["observaciones"] = df["observaciones"].str.replace("ND", "0")
    df["valor"] = df["observaciones"].str.replace(",", ".").astype('float').round(1)
    print(df.name + ": ")
    print(df[["observaciones", "fecha", "valor"]].head(1))

236000.CSV: 
                observaciones   fecha  valor
0                   787,10000  1/1964  787.1
232020.CSV: 
                observaciones   fecha    valor
0                 50657,00000  1/2004  50657.0
715100.CSV: 
               observaciones    fecha  valor
0                          0  12/1975    0.0
170000n.CSV: 
                observaciones   fecha   valor
0                  2995,77400  1/1996  2995.8


## Para terminar, exportar como ficheros CSV los dataframes tratados

In [24]:
# Pasar a CSV los ficheros necesarios
#Para sus nombres de fichero, almaceno en un dict los codigos y su traducción necesaria según la tabla ConstruData dada en la evaluación:
dict_traduccion = {
    "170000n.CSV": "PR-123",
    "555R20.CSV": "IVUS-211",
    "555R11.CSV": "IVUS-222",
    "232020.CSV": "VI-01",
    "236000.CSV": "CC-01",
    "715100.CSV": "RP-01",
    "715200.CSV": "CBS-232"
}
base_nombrefich = "indicadores_economicos"

nombres_ficheros = list()
for fichero in ficheros:
    nom_fichero = base_nombrefich + "_" + dict_traduccion[fichero] + ".csv"
    nombres_ficheros.append(nom_fichero)

for i in range(0, len(ficheros)):
    df_listado[i].to_csv(nombres_ficheros[i], sep="\t")

In [26]:
#Comprobar que los ficheros se han guardado correctamente
listado_pwd = os.listdir()
for fichero in nombres_ficheros:
    print(fichero in listado_pwd)

True
True
True
True
